# Análise Exploratória: Incompatibilidades nos Dados

**Objetivo**: Identificar todos os casos de incompatibilidades nos dados de absentismo para discussão com RH.

**Abordagem**: Trabalhar ao nível dos **códigos originais** (não níveis agregados) para evitar mascarar ou criar incompatibilidades artificiais.

## Metodologia:

1. **Criar Matriz de Compatibilidade**: Matriz 40x40 PRÉ-PREENCHIDA com regras automáticas
2. **Revisar e Ajustar**: Validar regras no Excel e ajustar se necessário
3. **Testar Incompatibilidades**: Usar matriz para identificar todos os casos reais no dataset
4. **Exportar Resultados**: Excel com detalhes (Data, Login, Nome, Códigos) para discussão com RH

## Regras Automáticas Aplicadas:

1. **Presença vs Ausência** (nível_1) → Incompatível
2. **Ausência Justificada vs Injustificada** → Incompatível
3. **Inatividade vs qualquer outro** → Incompatível
4. **Férias/Feriado/Folga vs trabalho** → Incompatível
5. **Doença vs Presença** → Incompatível
6. **Maternidade/Paternidade vs Presença** → Incompatível
7. **Formação vs Ausência Injustificada** → Incompatível

---

## 1. Carregamento de Dados

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Carregar códigos (mesma estrutura do notebook de apresentação)
print('Carregando códigos...')
codigos = pd.read_excel('códigos.xlsx', engine='openpyxl')
codigos.columns = ['codigo', 'descricao', 'nivel_1', 'nivel_2', 'nivel_3', 'nivel_4']

# Limpar espaços
for col in codigos.columns:
    if codigos[col].dtype == 'object':
        codigos[col] = codigos[col].str.strip()

print(f'✅ Códigos carregados: {len(codigos)} códigos')
print(f'   Níveis: N1={codigos["nivel_1"].nunique()}, N2={codigos["nivel_2"].nunique()}, N3={codigos["nivel_3"].nunique()}, N4={codigos["nivel_4"].nunique()}')

# Carregar dataset
print('\nCarregando dataset...')
df_raw = pd.read_csv('combined_data.csv', sep=',', parse_dates=['Data'])

# Merge com códigos para ter todos os níveis
df_raw = df_raw.merge(
    codigos[['codigo', 'nivel_1', 'nivel_2', 'nivel_3', 'nivel_4']], 
    left_on='segmento_processado_codigo', 
    right_on='codigo', 
    how='left'
)
df_raw.drop('codigo', axis=1, inplace=True)

print(f'\n✅ Dataset carregado: {len(df_raw):,} registros')
print(f'   Período: {df_raw["Data"].min().strftime("%Y-%m-%d")} a {df_raw["Data"].max().strftime("%Y-%m-%d")}')
print(f'   Colaboradores únicos: {df_raw["login_colaborador"].nunique():,}')
print(f'   Colunas com níveis: {[c for c in df_raw.columns if "nivel" in c]}')

---

## 2. Criação da Matriz de Compatibilidade entre Códigos

In [ ]:
# Obter lista de todos os códigos únicos
lista_codigos = sorted(codigos['codigo'].unique())
n_codigos = len(lista_codigos)

print('='*90)
print('📋 CRIAÇÃO DA MATRIZ DE COMPATIBILIDADE')
print('='*90)
print(f'\nTotal de códigos: {n_codigos}')
print(f'Tamanho da matriz: {n_codigos}x{n_codigos} = {n_codigos*n_codigos:,} combinações')

# Criar matriz de compatibilidade com regras de negócio automáticas
matriz_compat = pd.DataFrame(
    'Compatível',  # Valor padrão
    index=lista_codigos,
    columns=lista_codigos
)

print('\nAplicando regras de negócio automáticas...')

# Contar incompatibilidades aplicadas
count_incompativel = 0

# Iterar por todos os pares de códigos
for i, cod1 in enumerate(lista_codigos):
    info1 = codigos[codigos['codigo'] == cod1].iloc[0]
    
    for j, cod2 in enumerate(lista_codigos):
        if i >= j:  # Pular diagonal e parte inferior (matriz simétrica)
            continue
            
        info2 = codigos[codigos['codigo'] == cod2].iloc[0]
        
        incompativel = False
        
        # REGRA 1: Nível 1 - Presença vs Ausência (mutuamente exclusivas)
        if info1['nivel_1'] != info2['nivel_1']:
            # Presença incompatível com Ausência
            incompativel = True
        
        # REGRA 2: Nível 2 - Categorias incompatíveis
        elif info1['nivel_2'] == 'Ausência Justificada' and info2['nivel_2'] == 'Ausência Injustificada':
            incompativel = True
        elif info1['nivel_2'] == 'Ausência Injustificada' and info2['nivel_2'] == 'Ausência Justificada':
            incompativel = True
        
        # REGRA 3: Inatividade incompatível com tudo (exceto ela mesma)
        elif info1['nivel_2'] == 'Inatividade' or info2['nivel_2'] == 'Inatividade':
            incompativel = True
        
        # REGRA 4: Nível 3 - Férias/Feriado/Folga incompatível com trabalho
        elif 'Ferias / Feriado / Folga' in [info1['nivel_3'], info2['nivel_3']]:
            outro_n3 = info2['nivel_3'] if info1['nivel_3'] == 'Ferias / Feriado / Folga' else info1['nivel_3']
            if outro_n3 in ['Presença', 'Atraso / Saída Antecipada']:
                incompativel = True
        
        # REGRA 5: Doença incompatível com Presença normal
        elif 'Doença' in [info1['nivel_3'], info2['nivel_3']]:
            outro_n3 = info2['nivel_3'] if info1['nivel_3'] == 'Doença' else info1['nivel_3']
            if outro_n3 == 'Presença':
                incompativel = True
        
        # REGRA 6: Maternidade/Paternidade incompatível com Presença normal
        elif 'Maternidade / Paternidade' in [info1['nivel_3'], info2['nivel_3']]:
            outro_n3 = info2['nivel_3'] if info1['nivel_3'] == 'Maternidade / Paternidade' else info1['nivel_3']
            if outro_n3 == 'Presença':
                incompativel = True
        
        # REGRA 7: Formação incompatível com Ausência Injustificada
        elif 'Formação' in [info1['nivel_3'], info2['nivel_3']]:
            outro_n2 = info2['nivel_2'] if info1['nivel_3'] == 'Formação' else info1['nivel_2']
            if outro_n2 == 'Ausência Injustificada':
                incompativel = True
        
        # Aplicar à matriz (simétrica)
        if incompativel:
            matriz_compat.loc[cod1, cod2] = 'Incompatível'
            matriz_compat.loc[cod2, cod1] = 'Incompatível'
            count_incompativel += 1

print(f'\n✅ Matriz criada com {count_incompativel} pares marcados como "Incompatível"')
print(f'   Pares compatíveis: {n_codigos*(n_codigos-1)//2 - count_incompativel}')
print(f'\n💡 Regras aplicadas automaticamente:')
print(f'   1. Presença vs Ausência (nível 1)')
print(f'   2. Ausência Justificada vs Injustificada')
print(f'   3. Inatividade incompatível com tudo')
print(f'   4. Férias/Feriado/Folga vs trabalho')
print(f'   5. Doença vs Presença')
print(f'   6. Maternidade/Paternidade vs Presença')
print(f'   7. Formação vs Ausência Injustificada')
print(f'\n📝 Próximo passo: Revisar matriz no Excel e ajustar se necessário')

---

## 3. Exportação da Matriz para Excel

In [ ]:
# Exportar matriz para Excel
output_matriz = 'matriz_compatibilidade_codigos.xlsx'

with pd.ExcelWriter(output_matriz, engine='openpyxl') as writer:
    # Exportar matriz
    matriz_compat.to_excel(writer, sheet_name='Matriz_Compatibilidade', index=True)
    
    # Criar folha de referência com descrições dos códigos
    df_ref = codigos[['codigo', 'descricao', 'nivel_1', 'nivel_2', 'nivel_3', 'nivel_4']].copy()
    df_ref = df_ref.sort_values('codigo')
    df_ref.to_excel(writer, sheet_name='Referência_Códigos', index=False)
    
    # Criar folha de instruções
    instrucoes = pd.DataFrame({
        'Instruções': [
            '🎯 OBJETIVO: Validar e ajustar as regras de compatibilidade entre códigos',
            '',
            '✅ MATRIZ JÁ PRÉ-PREENCHIDA COM REGRAS AUTOMÁTICAS:',
            '   1. Presença vs Ausência → Incompatível',
            '   2. Ausência Justificada vs Injustificada → Incompatível',
            '   3. Inatividade vs qualquer outro → Incompatível',
            '   4. Férias/Feriado/Folga vs trabalho → Incompatível',
            '   5. Doença vs Presença → Incompatível',
            '   6. Maternidade/Paternidade vs Presença → Incompatível',
            '   7. Formação vs Ausência Injustificada → Incompatível',
            '',
            '📝 O QUE FAZER:',
            '   1. Revisar a folha "Matriz_Compatibilidade"',
            '   2. Verificar se as marcações "Incompatível" fazem sentido',
            '   3. ADICIONAR "Incompatível" onde faltar (casos não cobertos pelas regras)',
            '   4. MUDAR para "Compatível" se alguma regra estiver errada',
            '',
            '💡 DICAS:',
            '   - A matriz é SIMÉTRICA: se A+B é incompatível, B+A também é',
            '   - Diagonal (código consigo mesmo) é sempre Compatível',
            '   - Use "Referência_Códigos" para ver descrições',
            '',
            '⚠️ IMPORTANTE:',
            '   - Não mudar nome do ficheiro nem das folhas',
            '   - Usar apenas "Compatível" ou "Incompatível" (sem acentos)',
            '   - Após editar, GUARDAR e executar Secção 4 do notebook',
            '',
            f'📊 ESTATÍSTICAS DA MATRIZ:',
            f'   - Total de códigos: {n_codigos}',
            f'   - Pares marcados "Incompatível": {count_incompativel}',
            f'   - Pares marcados "Compatível": {n_codigos*(n_codigos-1)//2 - count_incompativel}'
        ]
    })
    instrucoes.to_excel(writer, sheet_name='INSTRUÇÕES', index=False)
    
    # Criar folha com lista dos pares incompatíveis (para facilitar revisão)
    pares_incomp = []
    for i, cod1 in enumerate(lista_codigos):
        for j, cod2 in enumerate(lista_codigos):
            if i < j and matriz_compat.loc[cod1, cod2] == 'Incompatível':
                desc1 = codigos[codigos['codigo'] == cod1]['descricao'].values[0]
                desc2 = codigos[codigos['codigo'] == cod2]['descricao'].values[0]
                pares_incomp.append({
                    'Código 1': cod1,
                    'Descrição 1': desc1,
                    'Código 2': cod2,
                    'Descrição 2': desc2,
                    'Regra': 'Automática'
                })
    
    df_pares = pd.DataFrame(pares_incomp)
    df_pares.to_excel(writer, sheet_name='Lista_Incompatíveis', index=False)

print(f'✅ Matriz exportada: {output_matriz}')
print(f'\n📋 Folhas criadas:')
print(f'   1. INSTRUÇÕES: Como usar e validar a matriz')
print(f'   2. Matriz_Compatibilidade: Matriz {n_codigos}x{n_codigos} PRÉ-PREENCHIDA')
print(f'   3. Lista_Incompatíveis: Lista dos {count_incompativel} pares incompatíveis')
print(f'   4. Referência_Códigos: Consulta de códigos e descrições')
print(f'\n🎯 PRÓXIMO PASSO:')
print(f'   1. Abrir: {output_matriz}')
print(f'   2. Revisar folha "Lista_Incompatíveis" (mais fácil que a matriz)')
print(f'   3. Se necessário, ajustar na "Matriz_Compatibilidade"')
print(f'   4. Guardar ficheiro')
print(f'   5. Executar Secção 4 do notebook')

---

## 4. Teste de Incompatibilidades (executar APÓS editar a matriz)

**⚠️ ATENÇÃO**: Só executar esta secção DEPOIS de editar e guardar `matriz_compatibilidade_codigos.xlsx`

In [ ]:
# Carregar matriz editada
print('Carregando matriz de compatibilidade editada...')
matriz_editada = pd.read_excel(
    'matriz_compatibilidade_codigos.xlsx',
    sheet_name='Matriz_Compatibilidade',
    index_col=0
)

# Contar incompatibilidades definidas
total_incompativeis = (matriz_editada == 'Incompatível').sum().sum()
# Dividir por 2 porque matriz é simétrica
total_incompativeis_unicos = total_incompativeis // 2

print(f'✅ Matriz carregada')
print(f'   Tamanho: {matriz_editada.shape[0]}x{matriz_editada.shape[1]}')
print(f'   Pares marcados como "Incompatível": {total_incompativeis} (ou {total_incompativeis_unicos} únicos)')

# Identificar dias com múltiplos registros
print('\nIdentificando dias com múltiplos registros...')
registros_por_dia = df_raw.groupby(['login_colaborador', 'Data']).size()
dias_duplicados = registros_por_dia[registros_por_dia > 1]

print(f'✅ Total de dias-colaborador únicos: {len(registros_por_dia):,}')
print(f'   Dias com MÚLTIPLOS registros: {len(dias_duplicados):,} ({len(dias_duplicados)/len(registros_por_dia)*100:.1f}%)')

# Filtrar apenas dias duplicados
dias_dup_list = dias_duplicados.index.tolist()
df_dup = df_raw.set_index(['login_colaborador', 'Data']).loc[dias_dup_list].reset_index()

print(f'   Dataset de dias duplicados: {len(df_dup):,} registros')

### 4.1. Teste de Todas as Incompatibilidades

In [ ]:
print('='*90)
print('🚨 TESTANDO INCOMPATIBILIDADES')
print('='*90)

# Dicionário para guardar resultados por par de códigos incompatíveis
resultados_incompatibilidades = {}

# Iterar por todos os dias duplicados
total_dias_testados = 0
total_incompatibilidades_encontradas = 0

for (login, data), count in dias_duplicados.items():
    total_dias_testados += 1
    
    # Obter códigos deste dia
    registros = df_dup[(df_dup['login_colaborador'] == login) & (df_dup['Data'] == data)]
    codigos_dia = registros['segmento_processado_codigo'].values
    nome = registros['nome_colaborador'].iloc[0]
    
    # Testar todos os pares de códigos
    for i, cod1 in enumerate(codigos_dia):
        for cod2 in codigos_dia[i+1:]:  # Evitar duplicatas e comparação consigo mesmo
            # Verificar na matriz se são incompatíveis
            if cod1 in matriz_editada.index and cod2 in matriz_editada.columns:
                compatibilidade = matriz_editada.loc[cod1, cod2]
                
                if compatibilidade == 'Incompatível':
                    total_incompatibilidades_encontradas += 1
                    
                    # Criar chave única para este par (ordenado alfabeticamente)
                    par = tuple(sorted([cod1, cod2]))
                    
                    # Adicionar ao dicionário de resultados
                    if par not in resultados_incompatibilidades:
                        resultados_incompatibilidades[par] = []
                    
                    resultados_incompatibilidades[par].append({
                        'Data': data,
                        'Login': login,
                        'Nome': nome,
                        'Codigo_1': cod1,
                        'Codigo_2': cod2
                    })

print(f'\n✅ Teste concluído')
print(f'   Dias testados: {total_dias_testados:,}')
print(f'   Incompatibilidades encontradas: {total_incompatibilidades_encontradas:,}')
print(f'   Pares de códigos incompatíveis encontrados: {len(resultados_incompatibilidades)}')

# Mostrar resumo dos pares incompatíveis encontrados
if len(resultados_incompatibilidades) > 0:
    print(f'\n📋 RESUMO DOS PARES INCOMPATÍVEIS ENCONTRADOS:')
    for par, casos in sorted(resultados_incompatibilidades.items(), key=lambda x: len(x[1]), reverse=True):
        cod1, cod2 = par
        desc1 = codigos[codigos['codigo'] == cod1]['descricao'].values[0]
        desc2 = codigos[codigos['codigo'] == cod2]['descricao'].values[0]
        print(f'\n   🔴 {cod1} + {cod2}: {len(casos)} casos')
        print(f'      {desc1}')
        print(f'      {desc2}')
else:
    print(f'\n✅ Nenhuma incompatibilidade encontrada nos dados!')

---

## 5. Exportação dos Resultados

In [ ]:
if len(resultados_incompatibilidades) == 0:
    print('⚠️ Nenhuma incompatibilidade encontrada. Nada para exportar.')
else:
    # Criar Excel com uma folha por par de códigos incompatíveis
    output_file = 'incompatibilidades_encontradas.xlsx'
    
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        # Criar folha de resumo
        resumo_data = []
        for par, casos in sorted(resultados_incompatibilidades.items(), key=lambda x: len(x[1]), reverse=True):
            cod1, cod2 = par
            desc1 = codigos[codigos['codigo'] == cod1]['descricao'].values[0]
            desc2 = codigos[codigos['codigo'] == cod2]['descricao'].values[0]
            resumo_data.append({
                'Código 1': cod1,
                'Descrição 1': desc1,
                'Código 2': cod2,
                'Descrição 2': desc2,
                'Casos': len(casos),
                'Folha': f'{cod1}+{cod2}'
            })
        
        df_resumo = pd.DataFrame(resumo_data)
        df_resumo.to_excel(writer, sheet_name='RESUMO', index=False)
        
        # Criar uma folha por par incompatível
        for par, casos in resultados_incompatibilidades.items():
            cod1, cod2 = par
            
            # Criar DataFrame com os casos
            df_casos = pd.DataFrame(casos)
            
            # Adicionar descrições dos códigos
            desc1 = codigos[codigos['codigo'] == cod1]['descricao'].values[0]
            desc2 = codigos[codigos['codigo'] == cod2]['descricao'].values[0]
            df_casos['Descrição_1'] = desc1
            df_casos['Descrição_2'] = desc2
            
            # Reordenar colunas
            df_casos = df_casos[['Data', 'Login', 'Nome', 'Codigo_1', 'Descrição_1', 'Codigo_2', 'Descrição_2']]
            
            # Nome da folha (máximo 31 caracteres)
            sheet_name = f'{cod1}+{cod2}'[:31]
            
            df_casos.to_excel(writer, sheet_name=sheet_name, index=False)
    
    print(f'✅ Resultados exportados: {output_file}')
    print(f'\n📋 Estrutura do ficheiro:')
    print(f'   - Folha RESUMO: {len(resultados_incompatibilidades)} pares incompatíveis')
    print(f'   - {len(resultados_incompatibilidades)} folhas com detalhes de cada par')
    print(f'\n📊 Total de casos: {total_incompatibilidades_encontradas:,}')
    print(f'\n🎯 Próximo passo: Analisar com RH e decidir ação para cada tipo de incompatibilidade')

---

## Resumo do Processo

### Workflow Completo:

1. ✅ **Carregamento de Dados**: Códigos e dataset
2. ✅ **Criação da Matriz**: Matriz 40x40 com todos os pares de códigos
3. ✅ **Exportação da Matriz**: `matriz_compatibilidade_codigos.xlsx`
4. 📝 **Edição Manual**: Marcar pares "Incompatíveis" no Excel
5. 🔍 **Teste de Incompatibilidades**: Identificar todos os casos reais nos dados
6. 📊 **Exportação de Resultados**: `incompatibilidades_encontradas.xlsx`

### Vantagens desta Abordagem:

- ✅ **Trabalha ao nível dos códigos originais** (não agregados)
- ✅ **Evita incompatibilidades artificiais** causadas pela agregação em níveis
- ✅ **Flexível**: Fácil adicionar/remover regras editando a matriz
- ✅ **Auditável**: Matriz explícita de todas as regras de compatibilidade
- ✅ **Reutilizável**: Matriz pode ser usada em análises futuras